# Description

Dataset: NSL-KDD (Trai 20 percent && Test+) using newest processing image method

Model: Resnet18

Method: Finetuning the convnet

Test case:
  - Image size: 10 feature
  - Default setting

[Tranfer Learning Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Link Colab of this file](https://colab.research.google.com/drive/126leoDf0tnL2y87dS2ClLPjcVqR7M1Qb)

# 1. Import zipped data

In [1]:
from google.colab import drive
drive.mount('/gdrive')
import os
import urllib.request
import zipfile
import shutil
l_rate=0.001
Summary_best_list=[]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy
# License: BSD
# Author: Sasank Chilamkurthy
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch import nn, optim
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader
from __future__ import print_function, division
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,confusion_matrix
import inspect
import zipfile
import pandas as pd
plt.ion()
from google.colab import files

In [ ]:
%cd /gdrive
%cd /
train_path='/gdrive/My Drive/KLTN/NSL-KDD 6 ftrs of Hiep/KDDtrain+_train.zip'
val_path='/gdrive/My Drive/KLTN/NSL-KDD 6 ftrs of Hiep/KDDtrain+_val.zip'
%cd /gdrive
%cd /
if os.path.exists('/content/data'): shutil.rmtree('/content/data')
zipfile.ZipFile(train_path).extractall()
#extracted folder: /content/NSL-KDD/data
zipfile.ZipFile(val_path).extractall()
os.rename('/content/data/KDDtrain+_val','/content/data/val')
os.rename('/content/data/KDDtrain+_train','/content/data/train')
data_dir = '/content/data'


# 2. Training

In [ ]:
%matplotlib inline

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")#or binary
    else:
        return metric_fn(true_y, pred_y,normalize=False)#set lai false
    
def get_scores(epoch,phase,p, r, f1, a, batch_size,dataset_size):
    lst=[]
    lst.append(epoch)
    lst.append(phase)
    for scores in (p, r, f1):
        lst.append(sum(scores)/batch_size)
    lst.append(sum(a)/dataset_size)
    return lst

In [ ]:
#@title
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            precision, recall, f1, accuracy = [], [], [], []
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                  acc.append(calculate_metric(metric, labels.data.cpu(), preds.cpu()))  
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            print("*"*100)
            score=get_scores(epoch,phase,precision, recall, f1, accuracy,len(dataloaders[phase]),dataset_sizes[phase])
            print("score:",score)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_acc_lst=score
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best val Acc Score: ',best_acc_lst)
    Summary_best_list.append(best_acc_lst)
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
num_classes=2
model_ft = models.mnasnet0_5(pretrained=True)
num_ftrs=model_ft.classifier[1].in_features
model_ft.classifier[1]=nn.Linear(num_ftrs, num_classes) 
print(model_ft)

In [ ]:
#@title
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=l_rate, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
torch.save(model.state_dict(), "mnasnet0_5.pt")

In [ ]:
!cp "mnasnet0_5.pt" "/gdrive/My Drive/KLTN/saved model 7 ftrs Hiep"